In [1]:
from bs4 import BeautifulSoup
import unidecode
import re
from sec_downloader import Downloader

import requests
from fake_useragent import UserAgent
import unicodedata
import html

In [2]:
def get_random_headers():
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    return headers

In [27]:
#soup = BeautifulSoup(get_html("AAPL", "10-K"))

C:\Users\310\AppData\Local\Temp\ipykernel_4428\812151145.py:1: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(get_html("AAPL", "10-K"))


In [97]:
url = "https://www.sec.gov/Archives/edgar/data/200406/000020040623000102/jnj-20231001.htm"

response = requests.get(url, headers=get_random_headers())
response.raise_for_status()  # Check that the request was successful

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

## Find all the table tags and decompose them to remove from the soup
#for table in soup.find_all('table'):
#    table.decompose()
#
## Now the soup object doesn't contain the tables, you can extract text
#text = soup.get_text(separator=' ', strip=True)

In [39]:
print(soup.title)

<title>jnj-20231001</title>


In [120]:
text = soup.get_text()
text = unidecode.unidecode(text)
text = re.sub(r'\s+', ' ', text).strip()

print(len(text), text[:1000])


294226 jnj-20231001000020040612/312023Q3FALSEhttp://fasb.org/us-gaap/2023#AccountingStandardsUpdate202204Member45600002004062023-01-022023-10-010000200406us-gaap:CommonStockMember2023-01-022023-10-010000200406jnj:A0.650NotesDue2024Member2023-01-022023-10-010000200406jnj:A5.50NotesDue2024Member2023-01-022023-10-010000200406jnj:A1.150NotesDue2028Member2023-01-022023-10-010000200406jnj:A1.650NotesDue2035Member2023-01-022023-10-0100002004062023-10-23xbrli:shares00002004062023-10-01iso4217:USD00002004062023-01-01iso4217:USDxbrli:shares00002004062023-07-032023-10-01xbrli:pure00002004062022-07-042022-10-0200002004062022-01-032022-10-0200002004062023-07-020000200406us-gaap:RetainedEarningsMember2023-07-020000200406us-gaap:AccumulatedOtherComprehensiveIncomeMember2023-07-020000200406us-gaap:CommonStockMember2023-07-020000200406us-gaap:TreasuryStockCommonMember2023-07-020000200406us-gaap:NoncontrollingInterestMember2023-07-020000200406us-gaap:RetainedEarningsMember2023-07-032023-10-010000200406u

In [122]:

pattern_mnda = r"(?i)item\s+\d+[A-Za-z]?\.?\s*(?:--|[-—])?\s*management[’'`]s\s+discussion\s+and\s+analysis\s+of\s+financial\s+condition\s+and\s+results\s+of\s+operations"


pattern_qnq = r"(?i)item\s+\d+[A-Za-z]?\.?\s*(?:--|[-—])?\s*quantitative\s+and\s+qualitative\s+disclosures\s+about\s+market\s+risk"

In [123]:
matches_mnda = re.findall(pattern_mnda, text)
matches_qnq = re.findall(pattern_qnq, text)
print(matches_mnda, '\n', matches_qnq)

["Item 2. Management's Discussion and Analysis of Financial Condition and Results of Operations", "Item 2 -- MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS"] 
 ['Item 3. Quantitative and Qualitative Disclosures About Market Risk', 'Item 3 -- QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK']


In [124]:
matches_mnda = re.finditer(pattern_mnda, text)
matches_qnq = re.finditer(pattern_qnq, text)

for i, mnda in enumerate(matches_mnda):
    print(f'mnda: {mnda.end()}')
for x, qnq in enumerate(matches_qnq):
    print(f'qnq: {qnq.start()}')
    
    
    #print(f'mnda №{i+1}:{mnda.span()} qnq №{x+1}: {qnq.span()}')

mnda: 96071
mnda: 240565
qnq: 96074
qnq: 287685


In [125]:
import re

def find_longest_substring(text, pattern_mnda, pattern_qnq):
    matches_mnda = list(re.finditer(pattern_mnda, text))
    matches_qnq = list(re.finditer(pattern_qnq, text))
    
    longest_substring = ""
    
    for mnda_match in matches_mnda:
        mnda_end = mnda_match.end()
        # Find the first matches_qnq after this mnda_match
        for qnq_match in matches_qnq:
            if qnq_match.start() > mnda_end:
                substring = text[mnda_end:qnq_match.start()]
                if len(substring) > len(longest_substring):
                    longest_substring = substring
                break  # We only want the first qnq match after each mnda match
    
    return longest_substring

# Example usage
longest = find_longest_substring(text, pattern_mnda, pattern_qnq)
print(f"The longest substring found is: '{len(longest)}'")


The longest substring found is: '47120'


In [116]:
import re

# Assuming 'text' is the variable containing the text extracted from your document
text = """
Item 2 — MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Item 10.    Management’s Discussion and Analysis of Financial Condition and Results of Operations

Item 2A.    Management's Discussion and Analysis of Financial Condition and Results of Operations.

Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations

ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Item 7.	
MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations
"""

text = unidecode.unidecode(text)
text = re.sub(r'\s+', ' ', text).strip()

print(len(text), text[:1000])





662 Item 2 -- MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Item 10. Management's Discussion and Analysis of Financial Condition and Results of Operations Item 2A. Management's Discussion and Analysis of Financial Condition and Results of Operations. Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations


In [118]:

pattern_mnda = r"(?i)item\s+\d+[A-Za-z]?\.?\s*(?:--|[-—])?\s*management[’'`]s\s+discussion\s+and\s+analysis\s+of\s+financial\s+condition\s+and\s+results\s+of\s+operations"


matches = re.findall(pattern_mnda, text)
print(matches)
print('\n')
print(len(matches) == 7)
print(len(matches))

["Item 2 -- MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", "Item 10. Management's Discussion and Analysis of Financial Condition and Results of Operations", "Item 2A. Management's Discussion and Analysis of Financial Condition and Results of Operations", "Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations", "ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", "Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", "Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations"]


True
7
